<a href="https://colab.research.google.com/github/dvarkless/InnopolisDS/blob/main/homeworks/Neural_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ некоторых параметров Полносвязной и Сверточной нейронной сети

In [ ]:
# System and fundamental stuff
import sys
from collections import OrderedDict
import functools
import math
import time
from itertools import product

# Types
from typing import Callable
from typing import OrderedDict as OrderedDictType
from types import FunctionType

# ML stuff
import numpy as np
from prettytable import PrettyTable
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchmetrics
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, Precision, Recall
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor

# Other
from alive_progress import alive_bar

## Надстройка для nn.Sequential

In [ ]:
class SeqModeler(nn.Sequential):
    def __init__(self, ord_dict: OrderedDictType, device='cpu') -> None:
        super().__init__(ord_dict)
        self.device = torch.device(device)
        self.to(self.device, non_blocking=True)

    def predict(self, X):
        X = self.forward(X)
        return torch.argmax(nn.Softmax(dim=1)(X), dim=1)


## Тренер нейронных сетей

In [ ]:
class Trainer:
    __defaults = {
        'batch_size': 100,
        'device': 'cpu',
        'epochs': 20,
        'criterion': nn.CrossEntropyLoss(),
        'enable_print': False,
        'metrics': None,
    }
    __must_have_params = ['model_class', 'model_params', 'set_optimizer']

    def __init__(self, **hp) -> None:
        self.config = self.__defaults.copy()
        for name, val in hp.items():
            self.config[name] = val

        for name in self.__must_have_params:
            if name not in self.config:
                print(f'Error: config parameter "{name}" is missing')
                sys.exit(1)

        self.model = self.config['model_class'](**self.config['model_params'].copy())
        opt_config = self.config['set_optimizer'].copy()
        opt_config['params'] = self.model.parameters()
        optimizer_name = opt_config.pop('name')
        self.optimizer = getattr(torch.optim, optimizer_name)(**opt_config)
        self.criterion = self.config['criterion']
        self.device = torch.device(self.config['device'])

    @property
    def data_batch(self):
        return self._data_batch

    @data_batch.setter
    def data_batch(self, data, /):
        if isinstance(data, torch.Tensor):
            self._data_batch = data.to(self.device, non_blocking=True).float()
        elif isinstance(data, np.ndarray):
            self.data_batch = torch.Tensor(data)
        else:
            raise ValueError(f'data of type {type(data)} is unacceptable')

    @property
    def targets_batch(self):
        return self._targets_batch

    @targets_batch.setter
    def targets_batch(self, targets):
        if isinstance(targets, torch.Tensor):
            self._targets_batch = targets.to(
                self.device, non_blocking=True)
        elif isinstance(targets, (np.ndarray, list, tuple)):
            self.targets_batch = torch.Tensor(targets)
        else:
            raise ValueError(f'data of type {type(targets)} is unacceptable')

    def fit(self, train_dataset, eval_dataset=None):
        train_dl = DataLoader(train_dataset, self.config['batch_size'], shuffle=True)
        for epoch in range(self.config['epochs']):
            avg_loss = []
            for (inputs, targets) in train_dl:
                self.data_batch, self.targets_batch = inputs, targets
                self.optimizer.zero_grad()
                yhat = self.model(self.data_batch)
                loss = self.criterion(yhat, self.targets_batch)
                avg_loss.append(loss)
                loss.backward()
                self.optimizer.step()
            avg_loss = torch.Tensor(avg_loss).mean()
            avg_loss.to(self.device)
            if self.config['enable_print']:
                print(
                    f'==========Epoch {epoch+1}/{self.config["epochs"]}==========')
                print(f'Loss: {avg_loss}')
                if self.config['metrics'] and eval_dataset:
                    metric_data = self.evaluate(eval_dataset)
                    for metric, data in zip(self.config['metrics'], metric_data):
                        print(f'{metric.__class__.__name__} = {data:.3f}')
        return self

    def evaluate(self, eval_dataset):
        eval_dl = DataLoader(eval_dataset, batch_size=10000)
        for data, targets in eval_dl:
            self.data_batch, self.targets_batch = data, targets
            predictions = self.model.predict(self.data_batch)
            metric_data = []
            for metric in self.config['metrics']:
                metric_data.append(metric(predictions, self.targets_batch))
            return tuple(metric_data)

    def predict(self, X):
        return self.model.predict(X)

## Интерфейс для перебора параметров моделей

In [ ]:
def timer(attr):
    """
        Декоратор используется для вывода времени,
        за которое выполняется метод класса
    """
    @functools.wraps(attr)
    def _wrapper(self, *args, **kwargs):
        start = time.perf_counter()
        result = attr(self, *args, **kwargs)
        runtime = time.perf_counter() - start
        print(f'{runtime:.3f}s')
        return result
    return _wrapper


class ModelRunner:
    """
        Класс предназначенный для удобного запуска моделей машинного обучения.

        Его возможности:
            - Создание экземпляров моделей с задаваемыми через словарь параметрами
              и их запуск через методы .fit() и .predict().
            - Вывод шкалы прогресса и времени выполнения методов моделей
            - Вывод различных метрик
            - Запуск одной модели с комбинацией различных параметров

        use case:
            >>> defaults = {'lr': 0.01, 'epochs': 100}
            >>> runner_inst = ModelRunner(ModelClass, timer=True, defaults=defaults, metrics=[accuracy])
            >>> runner_inst.run(training_data, eval_input, eval_answers, params={'lr': [0.001, 0.005], 'batch_size':[100],})

        inputs:
            model_class - Class of your model (not instance), all parameters should be passed through **kwargs
            defaults: dict - default kwargs for your model
            metrics: list - list of functions, they must take only two positional args: foo(preds, answers)
    """

    def __init__(self, model_class, defaults=None, metrics=None, responsive_bar=False) -> None:
        self.model_class = model_class
        self.metrics = metrics
        self._metric_data = []
        self._parameters_data = []
        if defaults is not None:
            self.defaults = defaults

        self._responsive_bar = responsive_bar

    def run(self, train, eval, params: dict, one_vs_one: bool = False):
        """
            Запустить проверку моделей с заданными данными и параметрами.

            Итерируемые параметры задаются в словаре params в виде:
                >>> params = {
                >>>     'lr': [1,2,3,4]
                >>>     'epochs': [100, 200]
                >>>     }
            Количество шагов проверки при этом зависит также от способа сочетания
            параметров:
                - При one_vs_one=True все доступные параметры сочетаются друг
                с другом, в данном примере получается 8 шагов

                - При one_vs_one=False параметры берутся по столбцам, при этом
                если в каком то списке не хватает значений, то берется его последнее
                значение в списке. В данном примере получается 4 шага

            inputs:
                train - training dataset, first column is answer labels
                eval_input - evaluation dataset without answers
                eval_answers - answer array in the same order as eval_input
                               size = (1, N)
                params - dict consisted of lists of the iterated parameters.
                        every value must be a list, even singular vals
                one_vs_one - parameters combination method, True is One vertus One;
                            False is columswise combination.

        """
        self._metric_data = []
        self._models = []
        curr_params = dict()
        if one_vs_one:
            # Проверка на наличие единственного значения в списке
            if len(list(params.values())) <= 1:
                pairs = list(*params.values())
            else:
                pairs = list(product(*list(params.values())))

            if self._responsive_bar:
                len_model_ticks = self.model_class(
                    self.defaults).define_tick(None, additive=len(eval))
            else:
                len_model_ticks = 1
            with alive_bar(len(list(pairs)*len_model_ticks), title=f'Проверка модели {self.model_class.__name__}', force_tty=True, bar='filling') as bar:
                # Распаковка параметров
                for vals in pairs:
                    for i, key in enumerate(params.keys()):
                        try:
                            curr_params[key] = vals[i]
                        except TypeError:
                            curr_params[key] = vals

                    print('-----With parameters-----')
                    for key, val in curr_params.items():
                        print(f'{key} = {val}')

                    self._parameters_data.append(list(curr_params.values()))
                    self._run_method(train, eval, curr_params, bar)
                    bar()  # продвижение полосы прогресса
        else:
            iter_lens = [len(val) for val in params.values()]
            if self._responsive_bar:
                len_model_ticks = self.model_class(
                    self.defaults).define_tick(None, additive=len(eval))
            else:
                len_model_ticks = 1
            max_len = max(iter_lens)
            with alive_bar(max_len*len_model_ticks, title=f'Проверка модели {self.model_class.__name__}', force_tty=True, bar='filling') as bar:
                for i in range(max_len):
                    for pos, key in enumerate(params.keys()):
                        this_len = iter_lens[pos]
                        try:
                            curr_params[key] = params[key][min(
                                this_len - 1, i)]
                        except TypeError:
                            curr_params[key] = params[key]

                    print('-----With parameters-----')
                    for key, val in curr_params.items():
                        print(f'{key} = {val}')

                    self._parameters_data.append(list(curr_params.values()))
                    self._run_method(train, eval, curr_params, bar)
                    bar()  # продвижение полосы прогресса

        print("===============RESULTS=================")
        pos = self._highest_metric_pos(self._metric_data)
        print(f'On iteration {pos}:')
        print(f"With hyperparameters: {self._parameters_data[pos]}")
        print(f'Got metrics: {self._metric_data[pos]}')

    def _run_method(self, train, eval, params: dict, bar_obj: Callable):
        """
            Внутренний обработчик ввода и вывода данных модели

            inputs:
                train - training dataset, first column is answer labels
                eval_input - evaluation dataset without answers
                eval_answers - answer array in the same order as eval_input
                               size = (1, N)
                params - dict of parameters that will be directly passed to the model


        """

        params_to_pass = self._mix_params(self.defaults, params)
        self.model = self.model_class(**params_to_pass)
    
        self.device = self.model.device

        if self._responsive_bar:
            self.model.define_tick(bar_obj, len(eval))

        print('~fit complete in ', end='')
        self._run_train(train, eval)

        print('~eval complete in ', end='')
        self._comma_metrics(eval)
        self._models.append(self.model)

    def _mix_params(self, main, invasive):
        """
            Внутренний метод для изменения словаря с параметрами

            Вносит изменения в основной словарь с параметрами 
            из другого словаря. Основной словарь при этом не меняется.

            inputs:
                main: dict - dict to be  inserted values into
                invasive: dict - mixed in values
            output - new dict with mixed values
        """
        maincpy = main.copy()
        for key, val in invasive.items():
            maincpy[key] = val
        return maincpy

    def _comma_metrics(self, evals):
        """
            Внутренний метод для получения метрик модели

            Можно в последствии получить все метрики через
            метод ModelRunner.get_metrics()

            inputs:
                evals: np.ndarray - labels
        """
        buff = []
        values = self._run_eval(evals)
        for metric, res in zip(self.metrics, values):
            if isinstance(metric, FunctionType):
                print(f"    {metric.__name__} = {res:.3f}")
            else:
                print(f"    {metric.__class__.__name__} = {res:.3f}")
            buff.append(res)
        self._metric_data.append(tuple(buff))

    def _highest_metric_pos(self, metrics):
        """
            Внутренний метод для получения позиции
            наибольшего значения метрик.

            Если видов метрик больше 1, то сравнивается их
            среднее геометрическое.

            inputs: 
                metrics: list - list of metrics, list of lists or
                list of floats
            output - index of the biggest value
        """
        score = [math.prod(vals) for vals in metrics]
        return score.index(max(score))

    def get_models(self):
        """
            Получить список со всеми использованными моделями

            output - list of all calculated models
        """

        return self._models

    def get_metrics(self):
        """
            Получить список со всеми значениями метрик

            Если метрик больше одной, то выдается список
            списков. Далее самим можно понять где какая метрика, 
            это не сложно

            output - list of all calculated metrics
        """
        return self._metric_data

    def get_params(self):
        """
            Получить список со всеми использованными
            гиперпараметрами

            Совпадает с тем, что передавалось в конструктор
            класса и в метод ModelRunner.run()

            output - list of hyperparameters
        """
        return self._parameters_data

    @timer
    def _run_train(self, train, eval):
        """
            Внутренний метод для запуска процесса 
            тренировки модели.

            inputs:
                train - training data
        """
        self.model.fit(train)

    @timer
    def _run_eval(self, eval_input) -> tuple:
        """
            Внутренний метод для получения ответов модели.

            inputs:
                eval_input - data to process
            output: tuple(torch.Tensor, torch.Tensor)
                targets, predictions

        """
        return self.model.evaluate(eval_input)
 

## Задаем начальные параметры
Датасет - CIFAR10

In [ ]:
    device = torch.device(
        'cuda') if torch.cuda.is_available() else torch.device('cpu')
    trans = Compose([ToTensor(), Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    train_dataset = CIFAR10('data/', train=True, download=True, transform=trans)
    val_dataset = CIFAR10('data/', train=False, download=True, transform=trans)
    model_params = OrderedDict([
        ('batch1', nn.BatchNorm2d(3)),
        ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
        ('relu1', nn.ReLU()),
        ('maxpool1', nn.MaxPool2d((2, 2))),

        ('batch2', nn.BatchNorm2d(16)),
        ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
        ('relu2', nn.ReLU()),
        ('maxpool2', nn.MaxPool2d((2, 2))),

        ('flatten3', nn.Flatten()),
        ('batch3', nn.BatchNorm1d(64*7*7)),
        ('linear3', nn.Linear(64*7*7, 100)),
        ('relu3', nn.ReLU()),
        ('linear4', nn.Linear(100, 10)),
        ('relu4', nn.ReLU()),
    ])

    optim_params = {
        'name': 'Adam',
        'params': None,
        'lr': 1e-3,
    }
    trainer_hp = {
        'batch_size': 50,
        'model_class': SeqModeler,
        'model_params': {'ord_dict': model_params, 'device': device},
        'set_optimizer': optim_params,
        'device': device,
        'criterion': nn.CrossEntropyLoss(),
        'enable_print': False,
        'metrics': [Accuracy(num_classes=10, average='macro').to(device), Recall(num_classes=10, average='macro').to(device), Precision(num_classes=10, average='macro').to(device)]
    }

Files already downloaded and verified
Files already downloaded and verified


### Тестирование полносвязной сети
##### Скорость обучения

In [ ]:
optim_params_1 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-2,
}
optim_params_2 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-3,
}

optim_params_3 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-4,
}

optim_params_4 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-5,
}
model_params_1 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(400, 100)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(100, 10)),
        ('relu3', nn.ReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}
model_params_2 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(400, 100)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(100, 10)),
        ('relu3', nn.ReLU()),
    ])
    
params2 = {'ord_dict': model_params_2, 'device': device}
model_params_3 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(400, 100)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(100, 10)),
        ('relu3', nn.ReLU()),
    ])

params3 = {'ord_dict': model_params_3, 'device': device}

model_params_4 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(400, 100)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(100, 10)),
        ('relu3', nn.ReLU()),
    ])

params4 = {'ord_dict': model_params_4, 'device': device}

params = {
    'model_params': [params1, params2, params3, params4],
    'set_optimizer': [optim_params_1, optim_params_2, optim_params_3, optim_params_4]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
fc_lr_table = PrettyTable()

fc_lr_table.add_column('Скорость обучения', ['1e-2', '1e-3', '1e-4', '1e-5'])
fc_lr_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(fc_lr_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=400, bias=True)), ('relu1', ReLU()), ('linear2', Linear(in_features=400, out_features=100, bias=True)), ('relu2', ReLU()), ('linear3', Linear(in_features=100, out_features=10, bias=True)), ('relu3', ReLU())]), 'device': device(type='cuda')}
on 0: set_optimizer = {'name': 'Adam', 'params': None, 'lr': 0.01}
on 0: ~fit complete in 224.582s
on 0: ~eval complete in 1.799s
on 0:     Accuracy = 0.100
on 0:     Recall = 0.100
on 0:     Precision = 0.010
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=400, bias=True)), ('relu1', ReLU()), ('linear2', Linear(in_features=400, out_features=100, bias=True)), ('relu2', ReLU()), ('linear3', Linear(in_features=100, out_features=10, bias=True)), ('relu

### Количество слоев

In [ ]:
model_params_1 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 10)),
        ('relu1', nn.ReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}
model_params_2 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 200)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(200, 10)),
        ('relu2', nn.ReLU()),
    ])
    
params2 = {'ord_dict': model_params_2, 'device': device}
model_params_3 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(400, 100)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(100, 10)),
        ('relu3', nn.ReLU()),
    ])

params3 = {'ord_dict': model_params_3, 'device': device}


params = {
    'model_params': [params1, params2, params3]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
fc_layers_table = PrettyTable()

fc_layers_table.add_column('Количество слоев', ['1', '2', '3'])
fc_layers_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(fc_layers_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=10, bias=True)), ('relu1', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 211.589s
on 0: ~eval complete in 1.861s
on 0:     Accuracy = 0.351
on 0:     Recall = 0.351
on 0:     Precision = 0.350
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=200, bias=True)), ('relu1', ReLU()), ('linear2', Linear(in_features=200, out_features=10, bias=True)), ('relu2', ReLU())]), 'device': device(type='cuda')}
on 1: ~fit complete in 213.130s
on 1: ~eval complete in 1.826s
on 1:     Accuracy = 0.506
on 1:     Recall = 0.506
on 1:     Precision = 0.514
on 2: -----With parameters-----
on 2: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1

### Количество нейронов в слоях

In [ ]:
model_params_1 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 100)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(100, 20)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(20, 10)),
        ('relu3', nn.ReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}
model_params_2 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(400, 100)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(100, 10)),
        ('relu3', nn.ReLU()),
    ])
    
params2 = {'ord_dict': model_params_2, 'device': device}
model_params_3 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 600)),
        ('relu1', nn.ReLU()),
        ('linear2', nn.Linear(600, 300)),
        ('relu2', nn.ReLU()),
        ('linear3', nn.Linear(300, 10)),
        ('relu3', nn.ReLU()),
    ])

params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
fc_neurons_table = PrettyTable()
fc_neurons_table.add_column('Количество нейронов', ['100 20', '400 100', '600 300'])
fc_neurons_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(fc_neurons_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=100, bias=True)), ('relu1', ReLU()), ('linear2', Linear(in_features=100, out_features=20, bias=True)), ('relu2', ReLU()), ('linear3', Linear(in_features=20, out_features=10, bias=True)), ('relu3', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 230.162s
on 0: ~eval complete in 1.816s
on 0:     Accuracy = 0.506
on 0:     Recall = 0.506
on 0:     Precision = 0.513
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=400, bias=True)), ('relu1', ReLU()), ('linear2', Linear(in_features=400, out_features=100, bias=True)), ('relu2', ReLU()), ('linear3', Linear(in_features=100, out_features=10, bias=True)), ('relu3', ReLU())]), 'device': device(type='cuda')}
on 1: ~fit complete in 

### Функции активации

In [ ]:
model_params_1 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('softplus1', nn.Softplus()),
        ('linear2', nn.Linear(400, 100)),
        ('softplus2', nn.Softplus()),
        ('linear3', nn.Linear(100, 10)),
        ('softplus3',nn.Softplus()), 
    ])
params1 = {'ord_dict': model_params_1, 'device': device}
model_params_2 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('sigmoid1', nn.Sigmoid()),
        ('linear2', nn.Linear(400, 100)),
        ('sigmoid2', nn.Sigmoid()),
        ('linear3', nn.Linear(100, 10)),
        ('sigmoid3', nn.Sigmoid()),
    ])
    
params2 = {'ord_dict': model_params_2, 'device': device}
model_params_3 = OrderedDict([
        ('flatten1',nn.Flatten()),
        ('linear1', nn.Linear(3072, 400)),
        ('tanh1', nn.Tanh()),
        ('linear2', nn.Linear(400, 100)),
        ('tanh2', nn.Tanh()),
        ('linear3', nn.Linear(100, 10)),
        ('tanh3', nn.Tanh()),
    ])

params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
fc_activations_table = PrettyTable()
fc_activations_table.add_column('Активации', ['softplus', 'sigmoid', 'tanh'])
fc_activations_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(fc_activations_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=400, bias=True)), ('softplus1', Softplus(beta=1, threshold=20)), ('linear2', Linear(in_features=400, out_features=100, bias=True)), ('softplus2', Softplus(beta=1, threshold=20)), ('linear3', Linear(in_features=100, out_features=10, bias=True)), ('softplus3', Softplus(beta=1, threshold=20))]), 'device': device(type='cuda')}
on 0: ~fit complete in 229.221s
on 0: ~eval complete in 1.899s
on 0:     Accuracy = 0.521
on 0:     Recall = 0.521
on 0:     Precision = 0.528
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('flatten1', Flatten(start_dim=1, end_dim=-1)), ('linear1', Linear(in_features=3072, out_features=400, bias=True)), ('sigmoid1', Sigmoid()), ('linear2', Linear(in_features=400, out_features=100, bias=True)), ('sigmoid2', Sigmoid()), ('linear3', Linear(in_features=100, out_fea

### Количество эпох

In [ ]:
model_params = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])

model_params_1 = {'ord_dict': model_params.copy(), 'device': device}
model_params_2 = {'ord_dict': model_params.copy(), 'device': device}
model_params_3 = {'ord_dict': model_params.copy(), 'device': device}
model_params_4 = {'ord_dict': model_params.copy(), 'device': device}
model_params_5 = {'ord_dict': model_params.copy(), 'device': device}

params = {
    'epochs': [5, 10, 25, 40, 80],
    'model_params': [model_params_1, model_params_2, model_params_3, model_params_4, model_params_5]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
fc_epochs_table = PrettyTable()
fc_epochs_table.add_column('Эпохи', ['5', '10', '25', '40', '80'])
fc_epochs_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(fc_epochs_table)
del model_runner

on 0: -----With parameters-----
on 0: epochs = 5
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu2', ReLU()), ('maxpool2', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=4096, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 61.617s
on 0: ~eval complete in 1.899s
on 0:     Accuracy = 0.470
on 0:     Recall = 0.470
on 0:     Precision = 0.369
on 1: -----With parameters-----
on 1: epochs = 10
on 1: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3,

## Сверточная нейронная сеть

### Скорость обучения

In [ ]:
model_params = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])

model_params_1 = {'ord_dict': model_params.copy(), 'device': device}
model_params_2 = {'ord_dict': model_params.copy(), 'device': device}
model_params_3 = {'ord_dict': model_params.copy(), 'device': device}
model_params_4 = {'ord_dict': model_params.copy(), 'device': device}
optim_params_1 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-2,
}
optim_params_2 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-3,
}

optim_params_3 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-4,
}

optim_params_4 = {
    'name': 'Adam',
    'params': None,
    'lr': 1e-5,
}

trainer_hp['model_params']['ord_dict'] = model_params


params = {
    'set_optimizer': [optim_params_1, optim_params_2, optim_params_3, optim_params_4],
    'model_params': [model_params_1, model_params_2, model_params_3, model_params_4]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_lr_table = PrettyTable()
conv_lr_table.add_column('Скорость обучения', ['1e-2', '1e-3', '1e-4', '1e-5'])
conv_lr_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_lr_table)
del model_runner

on 0: -----With parameters-----
on 0: set_optimizer = {'name': 'Adam', 'params': None, 'lr': 0.01}
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu2', ReLU()), ('maxpool2', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=4096, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 249.823s
on 0: ~eval complete in 1.839s
on 0:     Accuracy = 0.450
on 0:     Recall = 0.450
on 0:     Precision = 0.373
on 1: -----With parameters-----
on 1: set_optimizer = {'name': 'A

## Количество слоев

In [ ]:
model_params_1 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(16*16*16, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
    
params1 = {'ord_dict': model_params_1, 'device': device}

model_params_2 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params2 = {'ord_dict': model_params_2, 'device': device}
    
model_params_3 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 32, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('conv3', nn.Conv2d(32, 64, (2, 2), stride=1, padding=1)),
    ('relu3', nn.ReLU()),
    ('maxpool3', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*4*4, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3],
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_layers_table = PrettyTable()
conv_layers_table.add_column('Количество слоев', ['1', '2', '3'])
conv_layers_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_layers_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=4096, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 238.138s
on 0: ~eval complete in 1.879s
on 0:     Accuracy = 0.336
on 0:     Recall = 0.336
on 0:     Precision = 0.270
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1

### Количество нейронов

In [ ]:
model_params_1 = OrderedDict([
    ('conv1', nn.Conv2d(3, 8, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(8, 16, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(16*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}

model_params_2 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params2 = {'ord_dict': model_params_2, 'device': device}

model_params_3 = OrderedDict([
    ('conv1', nn.Conv2d(3, 64, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(64, 128, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(128*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3],
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_neurons_table = PrettyTable()

conv_neurons_table.add_column('Количество нейронов', ['8 16', '16 64', '64 128'])
conv_neurons_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_neurons_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 8, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu2', ReLU()), ('maxpool2', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=1024, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 253.296s
on 0: ~eval complete in 1.907s
on 0:     Accuracy = 0.312
on 0:     Recall = 0.312
on 0:     Precision = 0.201
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 

### Функции активации
Вместо софтплюс возьму LeakyRELU

In [ ]:
model_params_1 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.LeakyReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.LeakyReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('relu3', nn.LeakyReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.LeakyReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}

model_params_2 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('sigmoid1', nn.Sigmoid()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('sigmoid2', nn.Sigmoid()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('sigmoid3', nn.Sigmoid()),
    ('linear4', nn.Linear(100, 10)),
    ('sigmoid4', nn.Sigmoid()),
    ])
params2 = {'ord_dict': model_params_2, 'device': device}

model_params_3 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('tanh1', nn.Tanh()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('tanh2', nn.Tanh()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('tanh3', nn.Tanh()),
    ('linear4', nn.Linear(100, 10)),
    ('tanh4', nn.Tanh()),
    ])
params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3],
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_activations_table = PrettyTable()

conv_activations_table.add_column('Активации', ['LeakyReLU', 'sigmoid', 'tanh'])

conv_activations_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_activations_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', LeakyReLU(negative_slope=0.01)), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu2', LeakyReLU(negative_slope=0.01)), ('maxpool2', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=4096, out_features=100, bias=True)), ('relu3', LeakyReLU(negative_slope=0.01)), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', LeakyReLU(negative_slope=0.01))]), 'device': device(type='cuda')}
on 0: ~fit complete in 249.512s
on 0: ~eval complete in 1.907s
on 0:     Accuracy = 0.673
on 0:     Recall = 0.673
on 0:     Precision = 0.682
on 1: -----With parameters-----
on 1

### Количество эпох

In [ ]:
model_params = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])

model_params_1 = {'ord_dict': model_params.copy(), 'device': device}
model_params_2 = {'ord_dict': model_params.copy(), 'device': device}
model_params_3 = {'ord_dict': model_params.copy(), 'device': device}
model_params_4 = {'ord_dict': model_params.copy(), 'device': device}
model_params_5 = {'ord_dict': model_params.copy(), 'device': device}
params = {
    'epochs': [5, 10, 25, 40, 80],
    'model_params': [model_params_1, model_params_2, model_params_3, model_params_4, model_params_5]
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_epochs_table = PrettyTable()

conv_epochs_table.add_column('Эпохи', ['5', '10', '25', '40', '80'])

conv_epochs_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_epochs_table)
del model_runner

on 0: -----With parameters-----
on 0: epochs = 5
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu2', ReLU()), ('maxpool2', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=4096, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 62.962s
on 0: ~eval complete in 2.006s
on 0:     Accuracy = 0.463
on 0:     Recall = 0.463
on 0:     Precision = 0.333
on 1: -----With parameters-----
on 1: epochs = 10
on 1: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3,

### Изменение фильтра

In [ ]:
        # W_res = (W-F+2P)/S + 1
model_params_1 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (1, 1), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (1, 1), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*9*9, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}

        # W_res = (W-F+2P)/S + 1
model_params_2 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (3, 3), stride=1, padding=1)),
    ('sigmoid1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (3, 3), stride=1, padding=1)),
    ('sigmoid2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*8*8, 100)),
    ('sigmoid3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('sigmoid4', nn.ReLU()),
    ])
params2 = {'ord_dict': model_params_2, 'device': device}

        # W_res = (W-F+2P)/S + 1
model_params_3 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (4, 4), stride=1, padding=1)),
    ('tanh1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((2, 2))),

    ('conv2', nn.Conv2d(16, 64, (4, 4), stride=1, padding=1)),
    ('tanh2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((2, 2))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*7*7, 100)),
    ('tanh3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('tanh4', nn.ReLU()),
    ])
params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3],
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_filter_table = PrettyTable()

conv_filter_table.add_column('Размер фильтра', ['1x1', '3x3', '4x4'])

conv_filter_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_filter_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))), ('relu2', ReLU()), ('maxpool2', MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=5184, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 250.527s
on 0: ~eval complete in 1.805s
on 0:     Accuracy = 0.367
on 0:     Recall = 0.367
on 0:     Precision = 0.256
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(3, 3), stride=(1

### Размер слоя MaxPool2d

In [ ]:
        # W_res = (W-F+2P)/S + 1
model_params_1 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*34*34, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params1 = {'ord_dict': model_params_1, 'device': device}

        # W_res = (W-F+2P)/S + 1
model_params_2 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((3, 3))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((3, 3))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*4*4, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params2 = {'ord_dict': model_params_2, 'device': device}

        # W_res = (W-F+2P)/S + 1
model_params_3 = OrderedDict([
    ('conv1', nn.Conv2d(3, 16, (2, 2), stride=1, padding=1)),
    ('relu1', nn.ReLU()),
    ('maxpool1', nn.MaxPool2d((4, 4))),

    ('conv2', nn.Conv2d(16, 64, (2, 2), stride=1, padding=1)),
    ('relu2', nn.ReLU()),
    ('maxpool2', nn.MaxPool2d((4, 4))),

    ('flatten3', nn.Flatten()),
    ('linear3', nn.Linear(64*2*2, 100)),
    ('relu3', nn.ReLU()),
    ('linear4', nn.Linear(100, 10)),
    ('relu4', nn.ReLU()),
    ])
params3 = {'ord_dict': model_params_3, 'device': device}

params = {
    'model_params': [params1, params2, params3],
}

model_runner = ModelRunner(Trainer, trainer_hp, metrics=trainer_hp['metrics'])
model_runner.run(train_dataset, val_dataset, params)
metrics = model_runner.get_metrics()
conv_maxpool_table = PrettyTable()

conv_maxpool_table.add_column('Размер слоя MaxPool', ['1x1', '3x3', '4x4'])

conv_maxpool_table.add_column('Точность', list(map(lambda x: f'{float(x[0].detach().cpu().numpy()):.4f}', metrics)),)
print(conv_maxpool_table)
del model_runner

on 0: -----With parameters-----
on 0: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu2', ReLU()), ('flatten3', Flatten(start_dim=1, end_dim=-1)), ('linear3', Linear(in_features=73984, out_features=100, bias=True)), ('relu3', ReLU()), ('linear4', Linear(in_features=100, out_features=10, bias=True)), ('relu4', ReLU())]), 'device': device(type='cuda')}
on 0: ~fit complete in 242.093s
on 0: ~eval complete in 2.085s
on 0:     Accuracy = 0.100
on 0:     Recall = 0.100
on 0:     Precision = 0.010
on 1: -----With parameters-----
on 1: model_params = {'ord_dict': OrderedDict([('conv1', Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))), ('relu1', ReLU()), ('maxpool1', MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)), ('conv2', Conv2d(16, 64, kernel_size=(2, 2), stride=(1, 

#Вывод

In [ ]:
print('===== Полносвязная сеть =====')
print(fc_lr_table)
print(fc_layers_table)
print(fc_neurons_table)
print(fc_activations_table)
print(fc_epochs_table)
print('===== Свёрточная сеть =====')
print(conv_lr_table)
print(conv_layers_table)
print(conv_neurons_table)
print(conv_activations_table)
print(conv_epochs_table)
print(conv_filter_table)
print(conv_maxpool_table)

===== Полносвязная сеть =====
+-------------------+----------+
| Скорость обучения | Точность |
+-------------------+----------+
|        1e-2       |  0.1000  |
|        1e-3       |  0.5107  |
|        1e-4       |  0.5357  |
|        1e-5       |  0.5130  |
+-------------------+----------+
+------------------+----------+
| Количество слоев | Точность |
+------------------+----------+
|        1         |  0.3514  |
|        2         |  0.5056  |
|        3         |  0.5202  |
+------------------+----------+
+---------------------+----------+
| Количество нейронов | Точность |
+---------------------+----------+
|        100 20       |  0.5059  |
|       400 100       |  0.5099  |
|       600 300       |  0.5079  |
+---------------------+----------+
+-----------+----------+
| Активации | Точность |
+-----------+----------+
|  softplus |  0.5213  |
|  sigmoid  |  0.4477  |
|    tanh   |  0.4325  |
+-----------+----------+
+-------+----------+
| Эпохи | Точность |
+-------+----------+

Сверточная нейронная сеть показывает лучшие результаты при определенных параметрах.

В полносвязной сети изменения параметров не оказывают серьезного влияния на точность, однако стоит отметить что нейронная сеть недостаточно сложна для классификации изображений из датасета CIFAR10. Это видно в таблице зависимости точности от количества слоев, при большем количестве слоев точность выше.
- Функция активации Softplus, больше всего похожая на ReLU дает наивысшую точностью.
- Количества эпох меньше 80 недостаточно для полного обучения модели.

В сверточных нейронных сетях параметры сильнее влияют на модель. Можно отметить следующие моменты:
- Одного или двух сверточных слоев недостаточно для обучения (таб. Кол-во слоев)
- Функции активации, пропускающие отрицательные значения, оказывают негативное влияние на точность модели, как видно по функции Sigmoid из таблицы Активации.
- В данном случае сверточная сеть достигла максимальной точности при 5 эпохах обучения.
- Размер фильтра сверточного слоя лучше оставить 2х2 или 3х3.
- Тоже самое со слоем MaxPool